In [1]:
! wc -l /bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned/rank.txt

6980000 /bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned/rank.txt


In [2]:
! head /bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned/rank.txt

1048585	7187160	32.22602081298828
1048585	7187155	31.955062866210938
1048585	7187158	30.86893081665039
1048585	7187157	29.8802547454834
1048585	7617404	28.741893768310547
1048585	3856131	27.02429962158203
1048585	7187163	26.333473205566406
1048585	7187156	25.125255584716797
1048585	7546327	24.202661514282227
1048585	7617413	23.50704002380371


In [3]:
from collections import defaultdict
from tqdm.auto import tqdm
from beir.datasets.data_loader import GenericDataLoader
import random
print('lol')

lol


In [4]:
corpus, queries, qrels = GenericDataLoader("/bos/tmp15/adityasv/DomainAdaptation/MSMARCO/msmarco").load(split="dev")

In [5]:
qrels = defaultdict(set)
with open('/bos/tmp15/adityasv/DomainAdaptation/MSMARCO/msmarco/qrels/dev.tsv', 'r') as fi:
    fi.readline()
    for line in fi:
        qid, pid, label = line.strip().split('\t')

        label = int(label)
        if label > 0:
            qrels[qid].add(pid) 

In [6]:
len(qrels)

6980

In [17]:
new_coil_path = '/bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned_init_distilbert_base/rank.txt'
splade_path = '/bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight0_expansion-norm_max_expansion_weight_not_learned/rank.txt'
coil_path = '/bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight0_tokens_weight1_expansion-norm_max_expansion_weight_not_learned/rank.txt'
unicoil_path = '/bos/tmp15/adityasv/COILv2-vkeshava/model_unicoil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned_init_distilbert_base/rank.txt'
ensemble_path = '/bos/tmp15/adityasv/COILv2-vkeshava/interpolation_splade_coil/interpolation-5-rank.txt'

ensemble = defaultdict(dict)
with open(ensemble_path, 'r') as fi:
    for line in tqdm(fi):
        qid, pid, score = line.strip().split('\t')
        score = float(score)
        ensemble[qid][pid] = score

unicoil = defaultdict(dict)
with open(unicoil_path, 'r') as fi:
    for line in tqdm(fi):
        qid, pid, score = line.strip().split('\t')
        score = float(score)
        unicoil[qid][pid] = score

new_coil = defaultdict(dict)
with open(new_coil_path, 'r') as fi:
    for line in tqdm(fi):
        qid, pid, score = line.strip().split('\t')
        score = float(score)
        new_coil[qid][pid] = score

coil = defaultdict(dict)
with open(coil_path, 'r') as fi:
    for line in tqdm(fi):
        qid, pid, score = line.strip().split('\t')
        score = float(score)
        coil[qid][pid] = score

splade = defaultdict(dict)
with open(splade_path, 'r') as fi:
    for line in tqdm(fi):
        qid, pid, score = line.strip().split('\t')
        score = float(score)
        splade[qid][pid] = score

In [12]:
# ranking intesection: compute jaccard
def compute_jaccard_rankings(system1, system2, k_values=[1, 5, 10, 100, 1000]):
    result = {}
    for k in k_values:
        intersection = 0
        count = 0
        for qid in system1.keys():            
            system1_rank = sorted([(v,k) for k,v in system1[qid].items()], reverse=True)
            system1_rank = set(i for _,i in system1_rank[:k])

            system2_rank = sorted([(v,k) for k,v in system2[qid].items()], reverse=True)
            system2_rank = set(i for _,i in system2_rank[:k])

            assert len(system1_rank) == len(system2_rank)
            intersection += len(system1_rank.intersection(system2_rank))
            count += len(system2_rank)

        jaccard = intersection/count
        print(f"k: {k}, jaccard = {jaccard}")
        result[k] = jaccard
    
    return result        

In [18]:
print("splade, unicoil")
_ = compute_jaccard_rankings(splade, unicoil)
print('*'*10)
print("coil, unicoil")
_ = compute_jaccard_rankings(coil, unicoil)
print('*'*10)
print("coil, new_coil")
_ = compute_jaccard_rankings(coil, new_coil)
print('*'*10)
print("unicoil, new_coil")
_ = compute_jaccard_rankings(unicoil, new_coil)
print('*'*10)
print("ensemble, new_coil")
_ = compute_jaccard_rankings(ensemble, new_coil)
print('*'*10)
print("ensemble, unicoil")
_ = compute_jaccard_rankings(ensemble, unicoil)

splade, unicoil
k: 1, jaccard = 0.46117478510028653
k: 5, jaccard = 0.576647564469914
k: 10, jaccard = 0.5972922636103152
k: 100, jaccard = 0.5856045845272206
k: 1000, jaccard = 0.5549299426934098
**********
coil, unicoil
k: 1, jaccard = 0.4355300859598854
k: 5, jaccard = 0.5508022922636103
k: 10, jaccard = 0.575487106017192
k: 100, jaccard = 0.5597650429799427
k: 1000, jaccard = 0.5144631805157593
**********
coil, new_coil
k: 1, jaccard = 0.45558739255014324
k: 5, jaccard = 0.5751862464183382
k: 10, jaccard = 0.5980372492836676
k: 100, jaccard = 0.5841590257879656
k: 1000, jaccard = 0.5419283667621777
**********
unicoil, new_coil
k: 1, jaccard = 0.6832378223495702
k: 5, jaccard = 0.770458452722063
k: 10, jaccard = 0.7867765042979943
k: 100, jaccard = 0.7874255014326648
k: 1000, jaccard = 0.7697244985673353
**********
ensemble, new_coil
k: 1, jaccard = 0.5286532951289399
k: 5, jaccard = 0.6424355300859599
k: 10, jaccard = 0.6648137535816618
k: 100, jaccard = 0.6593094555873925
k: 1000,

In [8]:
splade_wins = set()
splade_losses = set()
for qid in splade.keys():
    for pid in qrels[qid]:
        if pid in splade[qid]:
            splade_wins.add((qid, pid))
        else:
            splade_losses.add((qid, pid))

            
coil_wins = set()
coil_losses = set()
for qid in coil.keys():
    for pid in qrels[qid]:
        if pid in coil[qid]:
            coil_wins.add((qid, pid))
        else:
            coil_losses.add((qid, pid))

new_coil_wins = set()
new_coil_losses = set()
for qid in new_coil.keys():
    for pid in qrels[qid]:
        if pid in new_coil[qid]:
            new_coil_wins.add((qid, pid))
        else:
            new_coil_losses.add((qid, pid))

In [10]:
len(splade_wins), len(splade_losses), len(coil_wins), len(coil_losses), len(new_coil_wins), len(new_coil_losses)

(7098, 339, 6980, 457, 7146, 291)

In [27]:
splade_better = splade_wins.intersection(new_coil_losses)
splade_worse = splade_losses.intersection(new_coil_wins)

coil_better = coil_wins.intersection(new_coil_losses)
coil_worse = coil_losses.intersection(new_coil_wins)

splade_over_coil = splade_wins.intersection(coil_losses)
coil_over_splade = coil_wins.intersection(splade_losses)

In [24]:
len(splade_better), len(splade_worse), len(coil_better), len(coil_worse)

(86, 134, 45, 211)

In [28]:
qid, pid = random.sample(splade_over_coil, k=1)[0]

print(qid, pid)
print("*"*50)
print(queries[qid])
print("*"*50)
print(corpus[pid])
print("@"*50)
for i,j in zip(sorted(new_coil[qid].items(), key=lambda x: x[1], reverse=True)[:10], sorted(splade[qid].items(), key=lambda x: x[1], reverse=True)[:10]):
    print("{:.2f}".format(i[1]), "{:.2f}".format(j[1]), i[0], j[0])
    print(corpus[i[0]])
    print(corpus[j[0]])
    
    print("#"*50)
print("*"*50)

657091 7661636
**************************************************
what else can marijuana be used for
**************************************************
{'text': 'Cannabis can be used by smoking, vaporization, within food, or as an extract. Cannabis is often used for its mental and physical effects, such as a high or stoned feeling, a general change in perception, euphoria (heightened mood), and an increase in appetite.', 'title': ''}
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
27.26 34.29 341674 7661636
{'text': 'Marijuana is used as a psychoactive (i.e. mind altering) recreational drug, for certain medical ailments and for religious and spiritual purposes. Sinsemilla, hash/hashish (resinous form) and hash oil (sticky black liquid) are stronger forms of marijuana.', 'title': ''}
{'text': 'Cannabis can be used by smoking, vaporization, within food, or as an extract. Cannabis is often used for its mental and physical effects, such as a high or stoned feeling, a general change in 

/tmp/ipykernel_59203/2770149936.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  qid, pid = random.sample(splade_over_coil, k=1)[0]
